<a href="https://colab.research.google.com/github/ooootaaaa/DeepLearning2022Late/blob/main/1129%5BSelenium%E3%81%A7%E7%94%BB%E5%83%8F%E5%8F%8E%E9%9B%862022%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##seleniumを使って自動で画像検索し、結果を収集する

##必要なアプリのインストール
###Seleniumのインストール(バージョンを指定する)

In [1]:
!pip install selenium==4.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 958 kB 6.6 MB/s 
     |████████████████████████████████| 140 kB 31.5 MB/s 
     |████████████████████████████████| 384 kB 31.4 MB/s 
     |████████████████████████████████| 57 kB 2.4 MB/s 
     |████████████████████████████████| 4.0 MB 14.9 MB/s 
     |████████████████████████████████| 58 kB 2.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.


###chromium-chromedriverのインストール

In [2]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 5 not upgraded.
Need to get 95.1 MB of archives.
After this operation, 319 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 107.0.5304.87-0ubuntu11.18.04.1 [1,158 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 107.0.5304.87-0ubuntu11.1

##seleniumを実行して操作する

###その他のライブラリのimport

In [3]:
import requests
from selenium import  webdriver
from selenium.webdriver.chrome.options import Options
import os
import time
import datetime
#エラー処理用
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


##ログ用に時刻を取得

In [4]:
tm_start = time.time()#処理時間計測用
dt_now = datetime.datetime.now()#現在時間
dt_date_str = dt_now.strftime('%Y/%m/%d %H:%M')#書式指定
print(dt_date_str)

2022/11/29 03:15


##検索に関する設定値を定義する

In [5]:
QUERY = '犬 フリー'#検索したいワード
LIMIT_DL_NUM = 100#ダウンロード数の上限
SAVE_DIR = 'img/dog_'#出力フォルダ(自動生成させる)
FILE_NAME = 'dog_'#ファイル名(後ろに連番と拡張子をつけていく)
TIMEOUT = 60 #検索のタイムアウト(秒)
ACCESS_WAIT = 1 #アクセスする間隔(秒)←超重要
RETRY_NUM = 3 #クリックやrequestのリトライ回数
DRIVER_PATH = '/usr/bin/chromedriver'#chromedriver.exeへのパス

##Chromeをヘッドレスモードで起動する
ヘッドレスモード = ブラウザを画面に開かないで実行するモード

In [6]:
# optionの指定を準備
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('start-fullscreen')
options.add_argument('--disable-plugins')
options.add_argument('--disable-extensions')
#起動する
driver= webdriver.Chrome(DRIVER_PATH,options=options)
driver.implicitly_wait(TIMEOUT)#タイムアウト指定
#ログに処理時間を出力
tm_driver = time.time()
print('WebDriver起動完了',f'{tm_driver - tm_start}s')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  # Remove the CWD from sys.path while we load stuff.


WebDriver起動完了 0.892305850982666s


##Googleの画像検索へアクセス

In [7]:
#QUERY条件付きでURLを定義する
url = f'https://www.google.com/search?q={QUERY}&tbm=isch'
#組み立てたURLでアクセスで検索
driver.get(url)
#時刻のログを取得
tm_geturl = time.time()
print('Google画像検索ページ取得',f'{tm_geturl - tm_driver:.1f}s')


Google画像検索ページ取得 2.0s


##検索結果の要素を取得していく

In [8]:
#ページのサムネイル画像の要素群を取得
#(islmpクラスが指定されたimg=サムネイルを全て拾う)
tmb_elems = driver.find_elements_by_css_selector('#islmp img')
#サムネイルのリストから各サムネイルのalt属性を取得
tmb_alts = [tmb.get_attribute('alt') for tmb in tmb_elems]
#alt属性が空(小さいサムネイル)のものを差し引いて合計を算出
count = len(tmb_alts) - tmb_alts.count('')
print(count)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  This is separate from the ipykernel package so we can avoid doing imports until


48


##下にスクロールさせてさらに検索結果を表示させてまたカウント

In [9]:
#定義したLIMIT_DL_NUM(ダウンロード数の上限)になるまでスクロールを繰り返す
while count < LIMIT_DL_NUM:
  #ページの一番下へスクロールして新しいサムネイル画像を表示させる
  driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
  time.sleep(1) #スリープを繰り返す間隔(秒)を入れる　←重要
  #(islmpクラスが指定されたimg=サムネイルを全て拾う)
  tmb_elems = driver.find_elements_by_css_selector('#islmp img')
  #サムネイルのリストから各サムネイルのalt属性を取得
  tmb_alts = [tmb.get_attribute('alt') for tmb in tmb_elems]
  #alt属性が空(小さいサムネイル)のものを差し引いて合計を算出
  count = len(tmb_alts) - tmb_alts.count('')
  print(count)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  import sys


100


##サムネイル画像をクリックすると右側に開かれる領域をオブジェクトに取得

In [10]:
imgframe_elem = driver.find_element_by_id(('islsp'))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """Entry point for launching an IPython kernel.


##画像へURLアクセスした保存する準備をする

In [14]:
#画像の保存フォルダを作成
os.makedirs(SAVE_DIR,exist_ok=True)
#HTTPリクエスト用のヘッダーを作る
HTTP_HEADERS = {'User-Aget':driver.execute_script('return navigator.userAgent;')}
print(HTTP_HEADERS)
#ダウンロードしたい対象のファイル拡張子
IMG_EXTS = ('.jpg','.jpeg','.png','.gif')

{'User-Aget': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/107.0.5304.87 Safari/537.36'}


##ダウンロード中に利用する関数を定義
###画像の拡張子を取得する関数

In [15]:
#拡張子を取得する関数
def get_exetension(url):
  url_lower = url.lower()
  for img_ext in IMG_EXTS:#拡張子のリストから1つずつ比較
    if img_ext in url_lower:
      extension = '.jpg' if img_ext == '.jpeg' else img_ext
      break
    #if終わり
    else:
      extension = ''
    #for終わり
  return extension
#関数終わり